# Cyclic Weight Transfer (CWT) with Client-Controlled Cyclic Workflow

In this example, we will demonstrate the Client-Controlled Cyclic Workflow using the Client API with the CIFAR10 dataset. 
This differs from the [Server-Controlled Cyclic Workflow](../cyclic/cyclic.ipynb), as the server is not involved in communication with sensitive information in the case that is it not trusted.

## Client-Controlled Cyclic Workflow

<img src="figs/cyclic_ccwf.png" alt="cyclic ccwf" width=35% height=35% />

The `CyclicServerController` is responsible for managing the lifecycle of the job, and will assign `cyclic_config` and `cyclic_start` tasks for configuration and to begin the training workflow. The configuration includes picking the starting client, result clients, and defining the cyclic order.

The `CyclicClientController` is responsible for the training logic once `cyclic_start` is sent, and the [Cyclic Workflow](../cyclic/cyclic.ipynb#cyclic_workflow) is algorithmically the same as the server-controlled version. The main difference is transferring the model is now encrypted with secure peer-to-peer messaging, and only the result clients receive the model, rather than the server.

See the [docs](https://nvflare.readthedocs.io/en/main/programming_guide/controllers/client_controlled_workflows.html#cyclic-learning) for more information about the Client-Controlled Cyclic Workflow.

## Converting DL training code to FL training code

We will be using the [Client API FL code](../code/fl/train.py) trainer converted from the original [Training a Classifer](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) example.

See [Converting to FL code using Client API](../sag/sag.ipynb#code) for more details.

## Job Configuration

The client configuration for the trainer with the Client API is standard with the `PTClientAPILauncherExecutor`, `SubprocessLauncher`, and our defined app script that supports the `train` task. 

We now add the additional executor `CyclicClientController` that handles all `cyclic_*` tasks and maps the `learn_task_name` to the `train` task implemented by the `PTClientAPILauncherExecutor`. The `PTFileModelPersistor` and `FullModelShareableGenerator` components are now also defined in the client configuration.

The server configuration is much simpler, with the `CyclicServerController` being set along with any configuration lifecycle arguments.

Let's use the Job CLI to create the job from a Client-Controlled Cyclic template:

In [ ]:
! nvflare config -jt ../../../../../job_templates

In [ ]:
! nvflare job create -j /tmp/nvflare/jobs/cyclic_cc_pt -w cyclic_cc_pt -sd ../code/fl -force

We can take a look at the server and client configurations and make any changes as desired:

In [ ]:
! cat /tmp/nvflare/jobs/cyclic_cc_pt/app/config/config_fed_server.conf

In [ ]:
! cat /tmp/nvflare/jobs/cyclic_cc_pt/app/config/config_fed_client.conf

Ensure that the `app_script` is set to the Client API FL `train.py` code and the model path for the persistor is set to `net.Net`.

## Prepare Data

Make sure the CIFAR10 dataset is downloaded with the following script:

In [ ]:
! python ../data/download.py

## Run the Job

Now we can run the job with the simulator:

In [ ]:
! nvflare simulator /tmp/nvflare/jobs/cyclic_cc_pt -w /tmp/nvflare/cyclic_cc_pt_workspace -t 2 -n 2 

Lastly, we have the [swarm](../swarm/swarm.ipynb) example, which covers swarm learning and client-controlled cross-site evaluation workflows.